Гайд по созданию веб-сайта "Mikoshi"

# 1) Начало

## 1.1) Создание Django проекта

Создаем Django проект `mikoshi` и сразу же приложение `main` 

In [ ]:
mkdir mikoshi
django-admin startproject mysite mikoshi
cd ./mikoshi
py manage.py startapp main

Настроим файл `settings.py`  
1) Подключим новое приложение `main` в `INSTALLED_APPS`

In [ ]:
INSTALLED_APPS = [
    ...
    'main',
]

2) Пропишем в `ALLOWED_HOSTS` ip адрес нашего VPS

In [ ]:
ALLOWED_HOSTS = ['your.ip.adress']

Внесем наш начальный шаблон сайта в приложение.  
1) Для этого создаем внутри приложения папку `templates` и в него сохраняем `list_char.html`
2) В том же приложение создаем папку `static` и в ней папки `css` и `js`, в которых сохраняем `ls.css` и `script.js` соотвественно

## 1.2) Выкладываем проект на Github

https://github.com/SenSheep/Mikoshi

## 1.3) Выгрузка на сервер

Подключаемся к VPS по SSH

In [ ]:
ssh user@ip

Переходим в нашу директорию, предаврительно создав её и выгружаем проект.

In [ ]:
mkdir /var/www/RED
cd /var/www/RED
git clone https://github.com/SenSheep/Mikoshi RED

Создаём виртуальное окружение и установливаем зависимости

In [ ]:
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt

Примененяем миграции и создаём суперпользователя

In [ ]:
python manage.py migrate
python manage.py createsuperuser

## 1.4) Настройка Gunicorn и Nginx

### 1.4.1) Gunicorn

Установлмваем Gunicorn

In [ ]:
pip install gunicorn

Создаём systemd-сервиса для Gunicorn

In [ ]:
sudo nano /etc/systemd/system/mikoshi.service

Добавляем:

In [ ]:
[Unit]
Description=Gunicorn instance to serve mikoshi
After=network.target

[Service]
User=www-data
Group=www-data
WorkingDirectory=/var/www/RED/mikoshi
ExecStart=/var/www/RED/venv/bin/gunicorn --workers 3 --bind unix:/var/www/RED/mikosh.sock --chdir /var/www/RED/mikoshi mysite.wsgi:application

Restart=always
RestartSec=5
Environment=PATH=/var/www/RED/venv/bin

[Install]
WantedBy=multi-user.target

Запуск и автозапуск:

In [ ]:
sudo systemctl start mikoshi
sudo systemctl enable mikoshi

### 1.4.2) Nginx

Устаналиваем Nginx

In [ ]:
sudo apt install nginx -y

Настраиваем и вписываем:

In [ ]:
sudo nano /etc/nginx/sites-available/mikoshi

In [ ]:
server {
    listen 80;
    server_name 195.133.18.185;

        location / {
        proxy_pass http://unix:/var/www/RED/mikosh.sock;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }
        location /static/ {
        alias /var/www/RED/mikoshi/staticfiles/;
}
}

Активируем конфиг:

In [ ]:
sudo ln -s /etc/nginx/sites-available/mikoshi /etc/nginx/sites-enabled
sudo nginx -t
sudo systemctl restart nginx

Чтобы Nginx увидел static, необходимо запустить `collectstatic`

In [ ]:
python manage.py collectstatic --noinput

# 2) Создание аутентификации пользователей